In [1]:
from services.url_extractor_service import UrlExtractor
from dotenv import load_dotenv
load_dotenv()
import os

api_key = os.getenv('SCRAPPING_TOOL_API_KEY')
url = "https://learn.microsoft.com/en-us/training/courses/ai-102t00"

exam_urls_list = UrlExtractor.scrape_course_modules_units_and_topics(url, api_key)

In [6]:
import json

exam_urls = json.loads(exam_urls_list)

In [33]:
topic_url = exam_urls['modules'][2]['units'][0]['topics'][4]['url']

In [53]:
name = exam_urls['modules'][2]['units'][0]['topics'][4]['name']

In [48]:
topic_url

'https://learn.microsoft.com/en-us/training/modules/analyze-text-ai-language/5-analyze-sentiment'

In [39]:

from langchain_community.document_loaders import ScrapingAntLoader

scrapingant_loader = ScrapingAntLoader(
    [topic_url],  # List of URLs to scrape
    api_key=api_key,  # Get your API key from https://scrapingant.com/
    continue_on_failure=True,  # Ignore unprocessable web pages and log their exceptions
)

In [42]:
docs = scrapingant_loader.load()

In [49]:
docs

[Document(metadata={'url': 'https://learn.microsoft.com/en-us/training/modules/analyze-text-ai-language/5-analyze-sentiment'}, page_content='Skip to main content\n\nThis browser is no longer supported.\n\nUpgrade to Microsoft Edge to take advantage of the latest features, security\nupdates, and technical support.\n\n[ Download Microsoft Edge ](https://go.microsoft.com/fwlink/p/?LinkID=2092881\n) [ More info about Internet Explorer and Microsoft Edge\n](https://learn.microsoft.com/en-us/lifecycle/faq/internet-explorer-microsoft-\nedge)\n\n[ Learn ](/en-us/) [ ](https://www.microsoft.com)\n\nSuggestions will filter as you type\n\nSign in\n\n  * [ Profile ](https://learn.microsoft.com/en-us/users/me/activity/)\n  * [ Settings ](https://learn.microsoft.com/en-us/users/me/settings/)\n\nSign out\n\n[ ](https://www.microsoft.com)\n\n[ Learn ](/en-us/)\n\n  * Discover\n\n    * [ Documentation ](/en-us/docs/)\n\nIn-depth articles on Microsoft developer tools and technologies\n\n    * [ Training

In [52]:
full_text = docs[0].page_content

In [68]:
filtered_text = full_text.partition(f"# {name}")

In [83]:
context = filtered_text[2].partition("## Next unit:")[0]

In [86]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
from langchain_core.tools import tool
load_dotenv()
import os


azure_openai_endpoint = "https://par-mcdv1i6v-eastus2.cognitiveservices.azure.com/"
azure_openai_key = "B5c8gMOhCs1BGSMAMkGl6qYaHi2S6kfvaabhTFRq7SmVT5g1qTweJQQJ99BFACHYHv6XJ3w3AAAAACOGrH1e"
deployment_name = "gpt-4.1-nano"
llm = AzureChatOpenAI(azure_endpoint=azure_openai_endpoint, 
                      api_key=azure_openai_key, 
                      api_version="2024-12-01-preview",
                      azure_deployment=deployment_name
                      )

In [90]:
prompt = f"""You're a smart assistant that helps users to prepare themselves to pass Microsoft certification exams.
You job is to generate practice questions for the user.

Use the following context to generate your sample questions.

### Context ###
{context}

Generate the questions and answers, each question have to have 3 answers, one correct and 2 incorrect, Or all of them correct, you decide.

Provide also the explanation for each answer and the source (if available).

Generate only 3 sample questions.

Level of difficulty of questions: basic.

"""

In [91]:

response = llm.invoke(prompt).content

print(response)

**Question 1:**  
What does sentiment analysis evaluate in a text document?  

A) The grammatical correctness of the text.  
B) Whether the text is positive, negative, or neutral. *(Correct)*  
C) The readability level of the text.  

**Explanation:**  
Sentiment analysis assesses the overall emotional tone or attitude expressed in a text, categorizing it as positive, negative, or neutral. It does not evaluate grammar or readability.  

**Source:**  
Based on the context provided, which describes how sentiment analysis evaluates positive or negative sentiments in text.

---

**Question 2:**  
In Azure AI Language sentiment analysis, what determines the overall sentiment of a document?  

A) The sentiment of the first sentence only.  
B) The sentiment classification of individual sentences and their combination. *(Correct)*  
C) The length of the document.  

**Explanation:**  
The overall document sentiment is derived from the classification of individual sentences: if all are neutral,